In [9]:
import torch

if(torch.cuda.is_available()):
    
    device = torch.device('gpu')
    
else:
    
    device = torch.device('cpu')

In [7]:
from app.assistant import ChatManager
from langchain_ollama import OllamaLLM
from retrieval.vector_store import get_vector_store

llm = OllamaLLM(model='gemma3:1b-it-fp16',base_url='http://localhost:11434/',temperature=0.3)




In [8]:
from langchain.memory import ConversationBufferMemory
from app.utils import contexts
from retrieval import retriever
from generation.prompt_templates import get_template  
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain


translator = get_template("translator")['translator']
top_label_intent = get_template("top_label_intent")['top_label_intent']
intent_mode = get_template("intent_mode")['intent_mode']



In [46]:
translation_prompt = """Translate this sentence from  Romanized Hindi to English.**Note:-** generate translation only do not explain.: Text:{query} \n
    Answer:"""

In [10]:
from langchain.prompts import PromptTemplate

# Modified PromptTemplate

translation_prompt_Temp = PromptTemplate(
    input_variables=['query'],
    template=translation_prompt
)

In [38]:
- name: "Alankar Digital Assistant"
- role: "You are Alankar, the official digital assistant for Alankar Flex Printing & Advertising Agency, developed and maintained by the Positive Pairs team at Eklavya University.You are an expert in Assisting Customers for queries related to Alankar shop printing and designing services.but you can also answer general personal or conversational questions like "Are you a bot?" or "How are you?" in a friendly, human-like way. Always respond kindly and clearly.You help customers explore and interact with the full range of digital printing services provided by the shop."

Note:
- ✅ Only generate responses. Do not repeat this prompt.
- ✅ Respond clearly, concisely, and ethically.

Task Scope:
- You must only answer queries from the following pieces of retrieved context.
- You may answer general knowledge questions when appropriate.
- Provide details about services and offerings at Alankar Flex Printing & Advertising Agency.
- Share information on promotional offers (product-wise and occasion-wise).
- Recommend suitable products (e.g., cards, photo frames, flex, logos, banners) based on user needs.
- Answer questions on design, customization, and materials used.
- ⚠️ If asked to place, track, or cancel an order, politely inform the user that it's not available via the assistant. Redirect to customer support or website.
- Provide business hours, location, contact, and shop-related info.
- Show sample product images if requested.
- ❌ Do not show price details in responses.

Uncertainty Policy:
- Only answer if you are 100% confident.
- If not sure, respond with: "I'm not confident enough to answer that."

Prohibited Actions:
🚫 Do NOT:
- Comment on recent news, politics, or trends.
- Help with hacking, cracking, or bypassing systems.
- Engage in or promote illegal, violent, or offensive content.
- Provide legal, financial, or medical advice.
- Express personal opinions or speculate.

Developer Info:
- Developed by: "Positive Pairs"
- Managed by: "Eklavya University AI Team"
- Version: "1.0.0"
- Last updated: "2025-04-17"

Contact Info:
- 📞 Contact1: +91 7389766105 
- 📞 Contact2: +91 9171453472
- 🌐 Website: https://alankar-ad.netlify.app

Farewell Behavior:
When the user says goodbye or ends the chat, respond warmly. Example responses:
- "Thank you for visiting Alankar Flex Printing & Advertising Agency! Have a great day 😊"
- "It was a pleasure assisting you! Feel free to come back anytime."
- "Goodbye! Reach out again if you need help with banners, posters, or anything else!"
- "Take care! I'm always here if you need help with your printing needs!"

Context: {context}
Response style:
Use a medium tone — not too brief, not too detailed. Focus on clarity and avoid over-explaining.

In [12]:
context = """The Positive Pairs Team, comprising of talented individuals from Eklavya University, Damoh, Madhya Pradesh, 
developed the Alankar Digital Assistant during their final year of Bachelor of Technology. This innovative digital assistant 
was designed using advanced technologies like prompt engineering, large language models (LLMs), and LangChain, 
to offer a dynamic and interactive user experience.

Team Members:
- Akash Prasad Mishra (Data Scientist): Hailing from Damoh, Akash played a key role in developing the assistant’s 
  data processing capabilities, ensuring it could efficiently understand and generate responses to a wide range of queries.

- Saumil Athya (Web Developer): Also from Damoh, Saumil worked on the web development aspect of the project, 
  ensuring the seamless integration and accessibility of the assistant across platforms.

Together, they utilized their BTech knowledge and skills to create a user-friendly, AI-powered assistant aimed at providing 
useful services to the university community. The assistant’s capabilities were enhanced through their deep understanding 
of LLMs and prompt engineering techniques, combined with the flexibility of LangChain for developing robust conversational models.

For any inquiries or additional information, you can reach Akash Prasad Mishra at 8878685316. 
The team’s project is a testament to their hard work and dedication, reflecting the cutting-edge potential of AI technology 
in transforming digital experiences."""

In [13]:
# Modified PromptTemplate
details_agent_prompt_Temp = PromptTemplate(
    input_variables=['query','context'],
    template=details_agent_prompt
)

In [39]:
prompt = translation_prompt_Temp

In [62]:
prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='Translate this sentence from  Romanized Hindi to English.**Note:-** generate translation only do not explain.: Text:{query} \n\n    Answer:')

'Do you do what you can?'

In [187]:
chain.prompt = translation_prompt

result = chain.run({'query':"""members of that team."""})

print('Translated_text:-',result)

chain.prompt = details_agent_prompt_Temp

result = chain.run({'context':context,'query':result.strip()})

result

Translated_text:- Member of that team.


'Thank you for your interest in Alankar Digital Assistant! We’re happy to help you explore our services.\n\nWe offer a wide range of digital printing solutions, including:\n\n*   **Flex Prints:** Perfect for signage, banners, and promotional materials.\n*   **Logo Design & Printing:** Create custom logos and high-quality prints.\n*   **Photo Frames & Wall Art:** Design and print stunning photo frames and wall art.\n*   **Business Cards & Stationery:** Design professional business cards and stationery.\n\nWe also have a selection of promotional products like cards and posters.\n\n**Promotional Offers:**\n\n*   **New Business Card Design:** Get 10% off your first business card design!\n*   **Special Offer:** 15% off all photo frame prints when you order before October 31st.\n\n**Hours:** We’re available Monday to Friday, 9 AM to 5 PM, Eastern Standard Time.\n\n**Location:** We’re located at [Insert Address Here].\n\n**Contact:** You can reach us at +91 7389766105 or 8878685316.\n\n**Samp

In [14]:
from retrieval.vector_store import get_vector_store



# Retrieve and generate using the relevant snippets of the blog.
retriever = get_vector_store().as_retriever()

C:\Users\mishr\PycharmProjects\Open-RAGA\retrieval\embedder.py:63: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [17]:
contextaware_q_system_prompt = """Given a chat history and the latest user question \
      which might reference context in the chat history, formulate a standalone question \
      which can be understood without the chat history. Do NOT answer the question, \
      just reformulate it if needed and otherwise return it as is."""

In [47]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory( k=3)


In [57]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableLambda, RunnableMap



# function to translate text from Romanized hindi to english


@staticmethod
def translate_text(input:str):

    chain = (translation_prompt_Temp | llm)
    
    result = chain.invoke({'query':input})
    
    return result.strip()
    
    


# history aware/context aware chain for retrieving the context from the vector database

def history_aware_retriever_chain():
    
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextaware_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    

    history_aware_retriever_chain = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )

    return history_aware_retriever_chain
    



# question answer chain for the answer  the  questions from the retrieved context/docs
def q_a_chain():
        
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", details_agent_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ])
    
    details_agent_chain = create_stuff_documents_chain(llm, qa_prompt)

    
    return details_agent_chain




question = 'give the shop contact number?'
    



# Step 1: Translate the question only
translate_chain = (translate_prompt | self.llm).invoke({'query': question}).strip()




 # create a rag pipeline that will combine the history chain and q_a_chain   

rag_chain  = create_retrieval_chain(history_aware_retriever_chain(), q_a_chain())




# invoke full chain 
result = full_chain.invoke({"question": translated_text.strip(), "chat_history": memory.buffer_as_messages})


memory.save_context({'inputs': translated_text.strip()},{'outputs':result['answer']})

print(result['answer'])



Give me the shop’s phone number?


In [56]:
full_chain.get_name()

'RunnableSequence'

In [52]:
print(result)

{'input': 'Give me the shop’s phone number?', 'chat_history': [HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}), AIMessage(content='Assistant: You’re very welcome! Take care and have a great day!', additional_kwargs={}, response_metadata={}), HumanMessage(content='can you place an order?', additional_kwargs={}, response_metadata={}), AIMessage(content='Assistant: Yes! You can ask me anything, anytime. I’m here to assist you at no cost.', additional_kwargs={}, response_metadata={}), HumanMessage(content='give the shop contact number?', additional_kwargs={}, response_metadata={}), AIMessage(content='Assistant: Alankar Flex Printing & Advertising Agency is located on Sagar Road, Damoh, Madhya Pradesh. You can reach us at +91 7389766105 or +91 9171453472. Our working hours are Monday to Sunday, 10:00 AM to 9:30 PM.', additional_kwargs={}, response_metadata={})], 'context': [Document(id='0b7674cd-575e-4d4a-a3e9-986e71dda56c', metadata={}, page_content='🏢 Business N

In [112]:
result

{'input': 'What is the location of the shop?',
 'chat_history': [],
 'context': [Document(id='874182d2-7446-4dd8-9959-eed886fe8b45', metadata={}, page_content='Order Context:\n\nCustomers can place orders by directly calling the shop owner at the phone number: +91 7389766105.\n\nThis phone number can also be used for:\n- Tracking existing orders\n- Cancelling previously placed orders\n- Communicating directly with the shop owner for any order-related queries\n\nAdditionally, customers can visit the shop in person to:\n- Place a new order\n- Check the status of an existing order\n- Cancel or modify their orders\n\nImportant Note:\nCurrently, there is no option available for placing or managing orders online. All order-related services are handled either through phone communication or offline visits to the shop.'),
  Document(id='f67164a1-0b01-4c38-ac42-51a379aa957b', metadata={}, page_content='User: Do you take orders online?\nAssistant: We currently assist customers through chat or pho

In [45]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "thanks?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

print(ai_msg_1["answer"])

second_question = "can you place an order?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})



print(ai_msg_2["answer"])



Assistant: You’re very welcome! Is there anything else I can help you with today?
Assistant: We currently assist customers through chat or phone. You can place your order by contacting the shop directly.


In [37]:
chat_history

[HumanMessage(content='tell me about the positive pairs team?', additional_kwargs={}, response_metadata={}),
 '“The Positive Pairs team, comprised of talented individuals from Eklavya University, is the driving force behind Alankar Flex Printing & Advertising Agency. They’re a dedicated group of individuals who developed the assistant using advanced technologies like prompt engineering, large language models (LLMs), and LangChain. It’s a really collaborative effort, with Akash Prasad Mishra, who’s a data scientist, and Saumil Athya, a web developer, playing key roles in bringing this innovative digital assistant to life.”']

In [31]:

ai_msg_2

{'input': 'how are you?',
 'chat_history': [HumanMessage(content='hi?', additional_kwargs={}, response_metadata={}),
  'Hello! 👋 I’m Alankar, the official digital assistant for Alankar Flex Printing & Advertising Agency. How can I assist you today?'],
 'context': [Document(id='a2456bfb-c063-4fe8-b2b6-0467552f2403', metadata={}, page_content='- Saumil Athya (Web Developer): Also from Damoh, Saumil worked on the web development aspect of the project, \n  ensuring the seamless integration and accessibility of the assistant across platforms.\n\nTogether, they utilized their BTech knowledge and skills to create a user-friendly, AI-powered assistant aimed at providing \nuseful services to the university community. The assistant’s capabilities were enhanced through their deep understanding \nof LLMs and prompt engineering techniques, combined with the flexibility of LangChain for developing robust conversational models.\n\nFor any inquiries or additional information, you can reach Akash Prasa

In [16]:
from retrieval.retriever import similar_retriever

In [19]:
similar_retriever('who is the owner of the shop')

"User: Good evening\nBot: Hi! You’re chatting with Alankar, here to assist with services like flex banners, glow sign boards, and more!\n\nUser: Hello, who am I speaking with?\nBot: Hey there! You’re speaking with Alankar, your digital guide to everything we offer at Alankar Flex Printing & Advertising Agency.\n\nUser: Hi Alankar\nBot: Hello and welcome to Alankar Flex Printing! I'm Alankar, your assistant. How can I help you today?\n\nUser: Hi Alankar\nBot: Greetings! I'm Alankar from Alankar Flex Printing & Advertising Agency. We offer everything from banners to birthday cards!\n\nUser: Is this the printing shop?\nBot: Hi! You’re chatting with Alankar, here to assist with services like flex banners, glow sign boards, and more!\n\nUser: Can you help me with some printing?\nBot: Hi there! This is Alankar from Alankar Flex Printing & Advertising Agency. I'm here to help with any printing needs you have."

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from scripts.prepare_data import get_file_contents
import yaml
import os
from pathlib import Path
from sentence_transformers import SentenceTransformer
from scripts.prepare_data import get_file_contents

In [26]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from scripts.prepare_data import get_file_contents
import yaml
import os
from pathlib import Path
from sentence_transformers import SentenceTransformer
from scripts.prepare_data import get_file_contents
import torch


root_path = os.path.dirname(os.path.abspath("__file__"))

# Load a .yaml or .yml file
with open("C:\\Users\mishr\PycharmProjects\Open-RAGA\config\dev\\model_config.yaml", "r",encoding="utf-8") as file:
    model_config = yaml.safe_load(file)



model_name = model_config['embedding_model']['name']  
model_path = model_config['embedding_model']['path'] 
model_keywords = model_config['embedding_model']['model_kwargs']
encode_kwargs = model_config['embedding_model']['encode_kwargs']


# Load a .yaml or .yml file
with open("C:\\Users\mishr\PycharmProjects\Open-RAGA\config\dev\\app_config.yaml", "r") as file:
    app_config = yaml.safe_load(file)


docs_path  = app_config['docs_path']['preprocessed']

chunk_size = 1000
chunk_overlap = 150
vectorstore_path = app_config['vector_store']['persist_path']
# docs path 
chat_path  = app_config['docs_path']['chat_history']








# get embeddings model

def download_embedding_model():
   pass
    



# get embeddings model


def get_embedding_model(model_path=model_path):

  if(os.path.exists(model_path)):


      embedding_model = HuggingFaceEmbeddings(
            model_name= model_path,
            model_kwargs=model_keywords,
            encode_kwargs=encode_kwargs   # important for cosine sim
        )



      return  embedding_model


  else:

      os.mkdir(model_path)       # make directory for embedding model

      embedding_model = SentenceTransformer(model_name)       # get model from sources

      embedding_model.save(model_path) # save the embedd. model 
      
      print("Embedding Model loaded Successfully..")
      
      return  embedding_model # return model for inferences

     
     




# ------------ updates the exsisitings vector database ---------------------------------------

def append_embeddings(docs_path= chat_path,vectorstore_path=vectorstore_path):

            # Split into chunks
            text_splitter = CharacterTextSplitter(chunk_size=chunk_size
                                                  , chunk_overlap=chunk_overlap)
            docs = text_splitter.create_documents(get_file_contents(docs_path))
            
            if(os.path.exists(docs_path)):
                
                    vectorstore  = FAISS.load_local(vectorstore_path, embeddings=get_embedding_model(),dangerous_deserialization=True)
                
                    # 3. Add new texts
                    vectorstore.add_texts(docs)
                    
                    # 4. Save updated index
                    vectorstore.save_local(vectorstore_path) 
    
                    print(f"Vector updated successfully in {vectorstore_path}:- ") 
            else:

                print(f"------>>>> No {docs_path} - path exits")
                


# --------------- create embeddings from the new docs -------------------------

def create_embeddings(docs_path= chat_path,vectorstore_path=vectorstore_path):

       if(os.path.exists(docs_path)):
               
           # Split into chunks
            text_splitter = CharacterTextSplitter(chunk_size=chunk_size
                                                  , chunk_overlap=chunk_overlap)
        
            docs = text_splitter.create_documents(get_file_contents(docs_path))  
        
            vectorstore  = FAISS.from_documents(docs, get_embedding_model())
     
            vectorstore.save_local(vectorstore_path)
     
            print(f"Vector added successfully in {vectorstore_path}:- ")

            return vectorstore
           
       else:
                       print(f"docs path not exists {vectorstore_path}:- ") 


           
       
                            
                        
        


In [27]:
create_embeddings(docs_path)

KeyboardInterrupt: 

NameError: name 'model_path' is not defined

In [ ]:
     
# Split into chunks
text_splitter = CharacterTextSplitter(chunk_size=chunk_size
                                      , chunk_overlap=chunk_overlap)

docs = text_splitter.create_documents(get_file_contents(docs_path))  

vectorstore  = FAISS.from_documents(docs, get_embedding_model())

vectorstore.save_local(vectorstore_path)



